In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Trading in V3
In this note, we discuss the idea of trading behind Uniswap V3. Given $S^* \in I_i$ and $\bar{S}^* \in I_k$ with
$l \leq k$ and $0 < S^* \leq \bar{S}^*$, we use the notations $(x_i,y_i)=R_3(I_i,L_i,S^*)$, $(x_k,y_k)=R_3(I_i,L_i,\bar{S}^*)$, $(0,y^*_j) = R_3(I_j,L_j,P_{j})$, $(x^*_j,0) = R_3(I_j,L_j,P_{j+1})$ for
$i\leq j\leq k$ and define the following quantities:
- **Amount of $Y$ reserves** between 
    $S^*$ and $\bar{S}^*$: $r_Y(S^* , \bar{S}^*) \equiv y_i+\sum^{k-1}_{j=i+1}y^*_j +y^*_k - y_k$
- **Amount of $X$ reserves** between
     $S^*$ and $\bar{S}^*$: $r_X(S^*, \bar{S}^*) \equiv x_k+ \sum^{k-1}_{j=i+1}x^*_j+ x^*_i-x_i $

The summation vanishes if $k=i$. One can write the liquidity as a step function of the pool price:
- $$L(S^*) \equiv \sum^\infty_{i=0} L_i I_{[P_i,P_{i+1}\;]}(S^*)$$
Then when $S^* \in [P_i , P_{i+1}]$, the reserves on that tick can be expressed as integrals:
- $$ x_i = L_i( \sqrt{S^*} - \sqrt{P}_i) = \int_{P_i}^{S^*} \frac{1}{2} L(t)t^{\frac{3}{2}} dt$$
- $$ y_i = L_i( \frac{1}{\sqrt{S^*}} - \frac{1}{\sqrt{P}_{i+1})} 
= \int_{S^*}^{P_{i+1}} \frac{1}{2} L(t)t^{-\frac{3}{2}} dt$$
Same for the quantities we defined above:
- $$r_X(S^* , \bar{S}^*) = \int_{S^*}^{\bar{S}^*} \frac{1}{2} L(t)t^{\frac{3}{2}} dt$$
- $$r_Y(S^* , \bar{S}^*) = \int_{S^*}^{\bar{S}^*} \frac{1}{2} L(t)t^{-\frac{3}{2}} dt$$

### Code

Parameters:
- ```S_pool```: The current pool price.
- ```tick_array```: An array that consists of tick prices. We expected it to be bounded away from zero.

Functions:
- ```activated_tick_index```: Given the current pool price, return the index of the tick where it locates. When the pool price equals to one of the tick price, choose the larger index (e.g $S^* = P_i$ , returns $i$).

In [17]:
def activated_tick_index(S_pool,tick_array):
    left, right = 0, len(tick_array) - 1
    while left <= right:
        mid = left + (right - left) // 2

        if tick_array[mid] <= S_pool < tick_array[mid + 1]:
            return mid  
        elif S_pool < tick_array[mid]:
            right = mid - 1
        else:
            left = mid + 1

    return -1  

Parameters:
- ```S_pool_0```: Lower pool price for computing $r_X$ and $r_Y$.
- ```S_pool_1```: Upper pool price for computing $r_X$ and $r_Y$.
- ```L_array```: An array that consists of the liquidity on each tick (corresponding to ```tick_array```). 

Functions:
- ```r_X```: Compute the amount of $X$ reserves between two given prices.
- ```r_Y```: Compute the amount of $Y$ reserves between two given prices.

In [17]:
def r_X(S_pool_0 , S_pool_1,L_array,tick_array):
    if S_pool_0 > S_pool_1:
        print("error!first argument should be smaller than the second!")
    if max(S_pool_0 , S_pool_1) > tick_array[-1]:
        print("error!outside of tick range!")
    index_0= activated_tick_index(S_pool_0,tick_array)
    index_1= activated_tick_index(S_pool_1,tick_array)
    
    if index_0 ==-1 or index_1 ==-1:
        print("error!outside of tick range!")
    amount =0
    for i in range(index_0 ,index_1+1):
        if i == index_0:
            amount+=L_array[i]*(np.sqrt(tick_array[i+1]-np.sqrt(S_pool_0) ))
        elif i == index_1:
            amount+= L_array[i]*( np.sqrt(S_pool_1) - np.sqrt(tick_array[i])  ) 
        else:
            amount+=L_array[i]*( np.sqrt(tick_array[i+1]) - np.sqrt(tick_array[i])  )
    return amount

def r_Y(S_pool_0 , S_pool_1,L_array,tick_array):
    if S_pool_0 > S_pool_1:
        print("error!first argument should be smaller than the second!")
    if max(S_pool_0 , S_pool_1) > tick_array[-1]:
        print("error!outside of price range!")
    index_0= activated_tick_index(S_pool_0,tick_array)
    index_1= activated_tick_index(S_pool_1,tick_array)
    
    if index_0 ==-1 or index_1 ==-1:
        print("error!outside of tick range!")
    amount =0
    for i in range(index_0 ,index_1+1):
        if i == index_0:
            amount+=L_array[i]*(1/np.sqrt(S_pool_0) - 1/np.sqrt(tick_array[i+1]) )
        elif i == index_1:
            amount+= L_array[i]*( 1/np.sqrt(tick_array[i]) - 1/np.sqrt(tick_array[i+1]) )  
        else:
            amount+=L_array[i]*( np.sqrt(tick_array[i]) - np.sqrt(S_pool_1)  )
    return amount

Parameters:
- ```delta_y```: The amount of asset $Y$ the trader wants to sell/buy. When the input is positive, the trader sells $Y$ to the pool. And when it is negative, the trader buys $Y$ from the pool.
- ```gamma```:fee constant in $ (0,1) $. 
- ```S_pool_ini```: The initial pool price when the trade is conducted.

Functions:
- ```trade```: Return a tuple consists of the amount of asset $X$ the trader receives/pays from the trade and the pool price after trade.

In [17]:
def trade(delta_y,gamma,S_pool_ini,L_array,tick_array):
    if delta_y>=0: #sell Y
        if gamma*delta_y > r_Y(tick_array[0], S_pool_ini,L_array,tick_array): #not enough x to pay the trader
            S_pool_final = tick_array[0]
            print("out of X reserves")
        else:    #enough x to pay
            sell=gamma*delta_y
            index_ini = activated_tick_index(S_pool_ini,tick_array)
            i=index_ini 
            S_pool = S_pool_ini
            while sell > 0: #trade tick-by-tick
                max_Y = r_Y(tick_array[i], S_pool,L_array,tick_array) #current max amount of Y on the tick                
                if sell <= max_Y: 
                    S_pool_fin=(1/np.sqrt(S_pool)+sell/L_array[i])**(-2)
                    break
                else: 
                    sell=sell-max_Y   #receive all X on that tick         
                    S_pool = tick_array[i]
                    i=i-1          #move to the next tick
                if sell<0 or i <0:
                    print("error")
        return r_X(S_pool_fin,S_pool_ini,L_array,tick_array) , S_pool_fin
    
    if delta_y<0: #buy y
        if delta_y > r_Y(S_pool_ini, tick_array[-1] ,L_array,tick_array): #not enough Y to pay the trader
            S_pool_final = tick_array[0]
            print("out of Y reserves")
        else:    #enough Y to pay
            buy=delta_y
            index_ini = activated_tick_index(S_pool_ini,tick_array)
            i=index_ini 
            S_pool = S_pool_ini
            while buy > 0: #trade tick-by-tick
                max_Y = r_Y(S_pool,tick_array[i+1],L_array,tick_array) #current max amount of Y on the tick                
                if buy <= max_Y: 
                    S_pool_fin=(1/np.sqrt(S_pool)- buy/L_array[i])**(-2)
                    break
                else: 
                    buy=buy-max_Y  
                    S_pool = tick_array[i+1]  
                    i=i+1   
                if buy<0 or i >len(tick_array)-1:
                    print("error")
        return gamma*r_X(S_pool_fin,S_pool_ini,L_array,tick_array) , S_pool_fin
    